### 아이디어
- 각 문자열의 위치의 타일이 한번이라도 교체되면 된다.
- 문자열이 매칭됐을때, 그곳의 위치와 매칭된 문자열의 길이를 같이 가져올 수 있으면 구현할 수 있을 것 같다.
- (챗봇) trie입력시 매칭하는 문자열의 길이 정보가 있다면 구할 수 있다. 단, 매칭가능한 문자가 여러개 있을 수 있다. 
  - 사실이라면, 어짜피 가장 긴 문자열이 이전 타일들을 전부 교체할 것이므로 여러개를 다 저장할 필요는 없을 것이다.

In [ ]:
import io, os, sys
sys.setrecursionlimit(10**5)
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

C_SIZE = 26
C_TO_I = lambda x: x - 97
class Trie:
  def __init__(self) :
    self.go = [None] * C_SIZE
    self.fail = None
    self.output = 0

  def add(self, k, n) :
    if not k :
      self.output = n
      return

    v = C_TO_I(k[0])
    if self.go[v] is None :
      self.go[v] = Trie()
    self.go[v].add(k[1:], n) 

from collections import deque 
class AhoCorasick:
  def __init__(self, trie: Trie, length: list, vis: list):
    self.trie = trie
    self.trie.fail = self.trie
    self.length = length
    self.vis = vis
    
    Q = deque([self.trie]) 
    while Q :
      u = Q.popleft()
      for i in range(C_SIZE) :
        v = u.go[i]
        if not v: continue
        
        if u == self.trie :
          v.fail = self.trie
        else :
          w = u.fail
          while w != self.trie and not w.go[i] :
            w = w.fail
          if w.go[i] :
            w = w.go[i]
          v.fail = w
        v.output += v.fail.output
        Q.append(v)
      
  def query(self, s: str):
    u = self.trie
    cnt = 0 
    for idx, c in enumerate(s) :
      i = C_TO_I(c)
      while u != self.trie and not u.go[i] :
        u = u.fail
      if u.go[i] :
        u = u.go[i]
      if u.output :
        start = idx - self.length[u.output] + 1
        end = start + self.length[u.output]
        for i in range(start, end):
          self.vis[i] = True
        cnt += 1
    return cnt

def sol() :
  N = int(input())
  trie = Trie()
  S = input().rstrip()

  M = int(input())
  P = [input().rstrip() for _ in range(M)]
  length = [0] 
  vis = [False] * N
  for i, s in enumerate(P) :
    trie.add(s, i+1)
    length.append(len(s)) #각 패턴의 길이를 저장
  
  aho = AhoCorasick(trie, length, vis)
  aho.query(S)

  print(aho.vis.count(False))

sol()

- 대충 정답을 출력하는 것 같긴 한데, 메모리 문제가 생겼다.
  - 트라이로 삼을 문자열의 개수와 길이가 각각 5000이라 그런것 같다..

In [ ]:
import io, os, sys
sys.setrecursionlimit(10**4)
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

C_SIZE = 26
C_TO_I = lambda x: x - 97
class Trie:
  def __init__(self) :
    self.go = [None] * C_SIZE
    self.fail = None
    self.output = 0

  def add(self, k, n) :
    if not k :
      self.output = n
      return

    v = C_TO_I(k[0])
    if self.go[v] is None :
      self.go[v] = Trie()
    self.go[v].add(k[1:], n) 

from collections import deque 
class AhoCorasick:
  def __init__(self, trie: Trie, length: list, vis: list):
    self.trie = trie
    self.trie.fail = self.trie
    self.length = length
    self.vis = vis
    
    Q = deque([self.trie]) 
    while Q :
      u = Q.popleft()
      for i in range(C_SIZE) :
        v = u.go[i]
        if not v: continue
        
        if u == self.trie :
          v.fail = self.trie
        else :
          w = u.fail
          while w != self.trie and not w.go[i] :
            w = w.fail
          if w.go[i] :
            w = w.go[i]
          v.fail = w
        v.output += v.fail.output
        Q.append(v)
      
  def query(self, s: str):
    u = self.trie
    cnt = 0 
    for idx, c in enumerate(s) :
      i = C_TO_I(c)
      while u != self.trie and not u.go[i] :
        u = u.fail
      if u.go[i] :
        u = u.go[i]
      if u.output :
        start = idx - self.length[u.output] + 1
        end = start + self.length[u.output]
        assert end <= len(self.vis)
        for i in range(start, end):
          self.vis[i] = True
        cnt += 1
    return cnt

def sol() :
  N = int(input())
  S = input().rstrip()

  M = int(input())
  P = [input().rstrip() for _ in range(M)]
  vis = [False] * N
  
  idx = 0
  while idx < M  : #패턴을 한번에 처리할 필요가 없으므로 100개마다 쿼리를 처리한다
    trie = Trie()
    length = [0]
    for i, s in enumerate(P[idx:]) :
      if i == 100: 
        idx += 100
        break
      trie.add(s, i+1)
      length.append(len(s)) #각 패턴의 길이를 저장
    else :
      aho = AhoCorasick(trie, length, vis)
      aho.query(S)
      break
    
    aho = AhoCorasick(trie, length, vis)
    aho.query(S)

  print(aho.vis.count(False))

sol()

- 쿼리를 100개씩 처리하는 아이디어. 하지만 IndexError가 어딘가에서 발생하고 있다.

In [ ]:
import io, os, sys
sys.setrecursionlimit(10**4)
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

C_SIZE = 26
C_TO_I = lambda x: x - 97
class Trie:
  def __init__(self) :
    self.go = [None] * C_SIZE
    self.fail = None
    self.output = 0

  def add(self, k, n) :
    if not k :
      self.output = n
      return

    v = C_TO_I(k[0])
    if self.go[v] is None :
      self.go[v] = Trie()
    self.go[v].add(k[1:], n) 

from collections import deque 
class AhoCorasick:
  def __init__(self, trie: Trie, length: list, vis: list):
    self.trie = trie
    self.trie.fail = self.trie
    self.length = length
    self.vis = vis
    
    Q = deque([self.trie]) 
    while Q :
      u = Q.popleft()
      for i in range(C_SIZE) :
        v = u.go[i]
        if not v: continue
        
        if u == self.trie :
          v.fail = self.trie
        else :
          w = u.fail
          while w != self.trie and not w.go[i] :
            w = w.fail
          if w.go[i] :
            w = w.go[i]
          v.fail = w
        if not v.output or self.length[v.output] < self.length[v.fail.output] : #가능한 가장 긴 길이의 타일로 업데이트한다
          v.output = v.fail.output
        Q.append(v)
      
  def query(self, s: str):
    u = self.trie
    cnt = 0 
    for idx, c in enumerate(s) :
      i = C_TO_I(c)
      while u != self.trie and not u.go[i] :
        u = u.fail
      if u.go[i] :
        u = u.go[i]
      if u.output :
        start = idx - self.length[u.output] + 1
        end = start + self.length[u.output]
        assert end <= len(self.vis)
        for i in range(start, end):
          self.vis[i] = True
        cnt += 1
    return cnt

def sol() :
  N = int(input())
  S = input().rstrip()

  M = int(input())
  P = [input().rstrip() for _ in range(M)]
  vis = [False] * N
  
  id = 0
  trie = Trie()
  length = [0]
  for s in P :
    if id == 50:  #모든 trie를 한번에 처리하지 않아도 되므로 100개씩 끊는다.
      aho = AhoCorasick(trie, length, vis)
      aho.query(S)
      id = 0
      trie = Trie()
      length = [0]

    id += 1
    trie.add(s, id)
    length.append(len(s)) #각 패턴의 길이를 저장
  
  aho = AhoCorasick(trie, length, vis)
  aho.query(S)

  print(aho.vis.count(False))

sol()

- 이번엔 TLE다. 쿼리 처리하는게 쿼리당 $\Omicron(N)$ 인게 문제인 것 같다.

In [ ]:
import io, os, sys
sys.setrecursionlimit(10**4)
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

C_SIZE = 26
C_TO_I = lambda x: x - 97
class Trie:
  def __init__(self) :
    self.go = [None] * C_SIZE
    self.fail = None
    self.output = 0

  def add(self, k, n) :
    if not k :
      self.output = n
      return

    v = C_TO_I(k[0])
    if self.go[v] is None :
      self.go[v] = Trie()
    self.go[v].add(k[1:], n) 

from collections import deque 
class AhoCorasick:
  def __init__(self, trie: Trie, length: list, vis: list):
    self.trie = trie
    self.trie.fail = self.trie
    self.length = length
    self.vis = vis
    
    Q = deque([self.trie]) 
    while Q :
      u = Q.popleft()
      for i in range(C_SIZE) :
        v = u.go[i]
        if not v: continue
        
        if u == self.trie :
          v.fail = self.trie
        else :
          w = u.fail
          while w != self.trie and not w.go[i] :
            w = w.fail
          if w.go[i] :
            w = w.go[i]
          v.fail = w
        if not v.output or self.length[v.output] < self.length[v.fail.output] : #가능한 가장 긴 길이의 타일로 업데이트한다
          v.output = v.fail.output
        Q.append(v)
      
  def query(self, s: str):
    u = self.trie
    for idx, c in enumerate(s) :
      i = C_TO_I(c)
      while u != self.trie and not u.go[i] :
        u = u.fail
      if u.go[i] :
        u = u.go[i]
      if u.output :
        start = idx - self.length[u.output] + 1
        self.vis[start] = max(self.vis[start], self.length[u.output]) #더 긴 타일이 있다면 업데이트한다.

def sol() :
  N = int(input())
  S = input().rstrip()

  M = int(input())
  P = [input().rstrip() for _ in range(M)]
  vis = [0] * N #i번째 위치에서 길이 vis[i]의 타일을 교체할 수 있다.
  
  id = 0
  trie = Trie()
  length = [0]
  for s in P :
    if id == 100:  #모든 trie를 한번에 처리하지 않아도 되므로 100개씩 끊는다.
      aho = AhoCorasick(trie, length, vis)
      aho.query(S)
      id = 0
      trie = Trie()
      length = [0]

    id += 1
    trie.add(s, id)
    length.append(len(s)) #각 패턴의 길이를 저장
  
  aho = AhoCorasick(trie, length, vis)
  aho.query(S)

  left = 0
  ans = [False] * N
  for i, v in enumerate(vis) : #순회하면서 교체할 수 있는 타일을 교체한 것으로 표시
    left = max(left, v)
    if left :
      ans[i] = True
    left -= 1
  print(ans.count(False))

sol()

- 쿼리를 나중에 한번에 $\Omicron(N)$ 으로 처리할 수 있도록 바꿨는데 여전히 다른 케이스에서 TLE가 난다. 뭐지??

In [ ]:
import io, os, sys
sys.setrecursionlimit(10**4)
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

C_SIZE = 26
C_TO_I = lambda x: x - 97
class Trie:
  def __init__(self) :
    self.go = [None] * C_SIZE
    self.fail = None
    self.output = 0

  def insert(self, s, i, idx) :
    k = C_TO_I(s[i])
    if self.go[k] is None :
      self.go[k] = Trie()
    if len(s)-1 == i :
      self.go[k].output = idx
    else :
      self.go[k].insert(s, i+1, idx)

from collections import deque 
class AhoCorasick:
  def __init__(self, trie: Trie, length: list, vis: list):
    self.trie = trie
    self.trie.fail = self.trie
    self.length = length
    self.vis = vis
    
    Q = deque([self.trie]) 
    while Q :
      u = Q.popleft()
      for i in range(C_SIZE) :
        v = u.go[i]
        if not v: continue
        
        if u == self.trie :
          v.fail = self.trie
        else :
          w = u.fail
          while w != self.trie and not w.go[i] :
            w = w.fail
          if w.go[i] :
            w = w.go[i]
          v.fail = w
        if not v.output or self.length[v.output] < self.length[v.fail.output] : #가능한 가장 긴 길이의 타일로 업데이트한다
          v.output = v.fail.output
        Q.append(v)
      
  def query(self, s: str):
    u = self.trie
    for idx, c in enumerate(s) :
      i = C_TO_I(c)
      while u != self.trie and not u.go[i] :
        u = u.fail
      if u.go[i] :
        u = u.go[i]
      if u.output :
        start = idx - self.length[u.output] + 1
        self.vis[start] = max(self.vis[start], self.length[u.output]) #더 긴 타일이 있다면 업데이트한다.

def sol() :
  N = int(input())
  S = input().rstrip()

  M = int(input())
  P = [input().rstrip() for _ in range(M)]
  vis = [0] * N #i번째 위치에서 길이 vis[i]의 타일을 교체할 수 있다.
  
  id = 0
  trie = Trie()
  length = [0]
  for s in P :
    if id == 100:  #모든 trie를 한번에 처리하지 않아도 되므로 100개씩 끊는다.
      aho = AhoCorasick(trie, length, vis)
      aho.query(S)
      id = 0
      trie = Trie()
      length = [0]

    id += 1
    trie.insert(s, 0, id)
    length.append(len(s)) #각 패턴의 길이를 저장
  
  aho = AhoCorasick(trie, length, vis)
  aho.query(S)

  left = 0
  ans = [False] * N
  for i, v in enumerate(vis) : #순회하면서 교체할 수 있는 타일을 교체한 것으로 표시
    left = max(left, v)
    if left :
      ans[i] = True
    left -= 1
  print(ans.count(False))

sol()

- trie의 구현을 바꿨더니 AC를 맞았다.. 구현을 손봐야겠다.

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

from collections import deque 
class Node:
  def __init__(self, s=None) :
    self.data = s
    self.go = {}
    self.fail = None
    self.output = 0

class AhoCorasick:
  def __init__(self) :
    self.root = Node()
  
  def insert(self, s: str, v) :
    u = self.root
    for c in s :
      if c not in u.go :
        u.go[c] = Node(c)
      u = u.go[c]
    u.output = v #문자열이 매칭됐을 때 그 문자열의 길이를 저장한다.
  
  def build(self) :
    Q = deque([self.root])
    while Q :
      u = Q.popleft()
      for k in u.go :
        v = u.go[k]
        if u == self.root :
          v.fail = self.root
        else :
          w = u.fail
          while w != self.root and k not in w.go :
            w = w.fail
          if k in w.go :
            w = w.go[k]
          v.fail = w
        
        if not v.output or v.output < v.fail.output : #가능한 가장 긴 길이의 타일로 업데이트한다
          v.output = v.fail.output
        Q.append(v)
  
  def query(self, s: str, vis: list) :
    u = self.root
    for i, c in enumerate(s) :
      while u != self.root and c not in u.go :
        u = u.fail
      if c in u.go : u = u.go[c]

      if u.output :
        start = i - u.output + 1
        vis[start] = max(vis[start], u.output) #더 긴 타일이 있다면 업데이트한다.

def sol() :
  N = int(input())
  S = input().rstrip()

  M = int(input())
  P = [input().rstrip() for _ in range(M)]
  vis = [0] * N #i번째 위치에서 길이 vis[i]의 타일을 교체할 수 있다.

  cnt = 0
  aho = AhoCorasick(vis)
  for i, s in enumerate(P) :
    if cnt == 50 :
      aho.build()
      aho.query(S, vis)
      aho = AhoCorasick(vis)
      cnt = 0

    cnt += 1
    aho.insert(s, len(s))
  aho.build()
  aho.query(S, vis)

  left = 0
  ans = [False] * N
  for i, v in enumerate(vis) : #순회하면서 교체할 수 있는 타일을 교체한 것으로 표시
    left = max(left, v)
    if left :
      ans[i] = True
    left -= 1
  print(ans.count(False))

sol()

- 뉴-템플릿.